In [1]:
import numpy as np

In [19]:
class Tensor:
    """
    Tensor
    """
    def __init__(self, numpy_array):
        if not isinstance(numpy_array, np.ndarray):
            raise ValueError("The class must be initialized with a Numpy array")
        self.value = numpy_array
        self.previous_tensors = None
        self.operation = None  # type of operation done to create this tensor.
        self.grad = None
        
    def backward(self):
        pass
    
    def compute_grad(self, tensor):
        if tensor.previous_tensors is None:
            pass
        else:
            pass
        
    def __add__(self, other):
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value + other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'add'
            new_tensor.previous_tensors = [self, other]
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
            
    
    def __mul__(self, other):
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value * other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'mul'
            new_tensor.previous_tensors = [self, other]
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
            
            
    
        
#     def __radd__(self, other):
#         return self.__add__(other)
    
#     def __sub__(self, other):
#         if isinstance(other, Tensor):
#             # check shapes
#             if not self.value.shape == other.value.shape:
#                 raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
#                                      format(str(self.value.shape), str(other.value.shape)))
#             operation_result = self.value - other.value
#             new_tensor = Tensor(operation_result)
#             new_tensor.operation = 'sub'
#             new_tensor.previous_tensors = [self, other]
#             return new_tensor
#         else:
#             raise ValueError("Operands must be of `Tensor` type.")
        
#     def __rsub__(self, other):
#         self.operation = ('rsub', other)
#         operation_result = other - self.value
#         new_tensor = Tensor(numpy_array=operation_result)
#         new_tensor.previous_tensor = self
#         return new_tensor
    
#     def __mul__(self, other):
#         self.operation = ('mul', other)
#         operation_result = self.value * other
#         new_tensor = Tensor(numpy_array=operation_result)
#         new_tensor.previous_tensor = self
#         return new_tensor
    
#     def mean(self):
#         self.operation = ('mean', None)
#         operation_result = self.value.mean()
#         new_tensor = Tensor(numpy_array=operation_result)
#         new_tensor.previous_tensor = self
#         return new_tensor
    
    def __str__(self):
        return self.value.__repr__()

In [27]:
arr1 = np.random.rand(2, 3)
arr2 = np.random.rand(2, 3)

In [28]:
t1 = Tensor(arr1)
t2 = Tensor(arr2)
print('t1:', t1)
print('t2:', t2)

t1: array([[0.16062265, 0.33623246, 0.43900086],
       [0.08721481, 0.23911905, 0.78727217]])
t2: array([[4.81832982e-04, 3.01818775e-01, 5.88160485e-01],
       [9.93574224e-01, 8.59893487e-01, 2.02196292e-01]])


In [30]:
t3 = t1 / t2
print(t3)

TypeError: unsupported operand type(s) for /: 'Tensor' and 'Tensor'

In [23]:
t4 = t2 + t1
print(t4)

array([[0.91545703, 0.43038079, 0.40962668],
       [0.48922999, 0.15241042, 1.17075516]])
